In [32]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import csv

In [33]:
batch_size=64
csv_flie="atlas-higgs-challenge-2014-v2.csv"

In [42]:
Data=np.array(pd.read_csv("atlas-higgs-challenge-2014-v2.csv"))

In [34]:
print(data[0][1:-3])
datas=data[1][1:-3].astype('float64')
print(datas)
compose=transforms.Compose(transforms.ToTensor())
print(data.shape)
print(type(data))
print(data.dtype)
print(torch.from_numpy(datas))

[138.47 51.655 97.82700000000001 27.98 0.91 124.711 2.6660000000000004
 3.0639999999999996 41.928000000000004 197.76 1.5819999999999999 1.396 0.2
 32.638000000000005 1.0170000000000001 0.381 51.626000000000005 2.273
 -2.414 16.824 -0.27699999999999997 258.733 2 67.435 2.15 0.444 46.062
 1.24 -2.475 113.49700000000001 0.00081448039868]
[ 1.60937000e+02  6.87680000e+01  1.03235000e+02  4.81460000e+01
 -9.99000000e+02 -9.99000000e+02 -9.99000000e+02  3.47300000e+00
  2.07800000e+00  1.25157000e+02  8.79000000e-01  1.41400000e+00
 -9.99000000e+02  4.20140000e+01  2.03900000e+00 -3.01100000e+00
  3.69180000e+01  5.01000000e-01  1.03000000e-01  4.47040000e+01
 -1.91600000e+00  1.64546000e+02  1.00000000e+00  4.62260000e+01
  7.25000000e-01  1.15800000e+00 -9.99000000e+02 -9.99000000e+02
 -9.99000000e+02  4.62260000e+01  6.81041907e-01]
(818238, 35)
<class 'numpy.ndarray'>
object
tensor([ 1.6094e+02,  6.8768e+01,  1.0323e+02,  4.8146e+01, -9.9900e+02,
        -9.9900e+02, -9.9900e+02,  3.4730

In [35]:
class Dataset_Cust(torch.utils.data.Dataset):
    def __init__(self, csv_file, train=True, rate_train=0.9, transformation=None):
        'Initialization'
        self.data=np.array(pd.read_csv(csv_file))
        self.train=train
        self.rate_train=rate_train
        self.numpoint=len(self.data)
        self.numtrain=int(0.9*self.numpoint)
        self.numtest=self.numpoint - self.numtrain
        self.transformation=transformation
        

    def __len__(self):
        'Denotes the total number of samples'
        if self.train:
            return self.numtrain
        else:
            return self.numtest
    

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        if not self.train:
            index=index+self.numtrain
        
        sample=torch.from_numpy(self.data[index][1:-4].astype('float64'))
        label=self.data[index][-3]
        
        if self.transformation :
            sample=self.transformation(sample)

        return sample, label
    
    def Set_Train(train):
        self.train=train

In [36]:
Dataset=Dataset_Cust(csv_flie)

In [37]:
Train_Loader=torch.utils.data.DataLoader(Dataset, batch_size=batch_size, shuffle=True)

In [44]:
for i, sample in enumerate(Train_Loader,0):
        inputs, labels= sample
        print(inputs)
        print(labels)
        if(i >10):
            break

tensor([ 1.3847e+02,  5.1655e+01,  9.7827e+01,  2.7980e+01,  9.1000e-01,
         1.2471e+02,  2.6660e+00,  3.0640e+00,  4.1928e+01,  1.9776e+02,
         1.5820e+00,  1.3960e+00,  2.0000e-01,  3.2638e+01,  1.0170e+00,
         3.8100e-01,  5.1626e+01,  2.2730e+00, -2.4140e+00,  1.6824e+01,
        -2.7700e-01,  2.5873e+02,  2.0000e+00,  6.7435e+01,  2.1500e+00,
         4.4400e-01,  4.6062e+01,  1.2400e+00, -2.4750e+00,  1.1350e+02],
       dtype=torch.float64)



[100000 138.47 51.655 97.82700000000001 27.98 0.91 124.711
 2.6660000000000004 3.0639999999999996 41.928000000000004 197.76
 1.5819999999999999 1.396 0.2 32.638000000000005 1.0170000000000001 0.381
 51.626000000000005 2.273 -2.414 16.824 -0.27699999999999997 258.733 2
 67.435 2.15 0.444 46.062 1.24 -2.475 113.49700000000001 0.00081448039868
 's' 't' 0.0026533113373299996]



tensor([[ 8.0678e+01,  1.4917e+01,  5.6583e+01,  ..., -9.9900e+02,
         -9.9900e+02,  0.0000e+00],
        [ 1.2744e+02,  7.9670e+01,  8.3048e+01,  